<a href="https://colab.research.google.com/github/Akanksha131313/AI_Projects_Journey/blob/main/VGG16_Cats_vs_Dogs_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Dataset Explanation (Short):**

The dataset contains images of cats and dogs.
Each image belongs to one of two categories — "cats" or "dogs".
We’ll use ImageDataGenerator for data augmentation and resizing all images to 150×150 pixels — which is the required input size for VGG16

###**1) Utilize VGG16 for Feature Extraction-**

####**a) Employ the VGG16 model, excluding its top layers, to serve as a feature extractor for cat and dog images-**

In [7]:
# Import libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import VGG16

# Load Cats vs Dogs dataset
(raw_train, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

# Set image size as required by VGG16
IMG_SIZE = 150

# Resize and normalize images (0–1 range)
def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train = raw_train.map(format_image)
test = raw_test.map(format_image)

# Batch data for training
BATCH_SIZE = 32
train_batches = train.shuffle(1000).batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

# Load pre-trained VGG16 (without top layers)
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

# Freeze convolutional layers to retain pre-trained features
for layer in vgg_base.layers:
    layer.trainable = False

print("VGG16 base model loaded successfully as feature extractor.")


VGG16 base model loaded successfully as feature extractor.


####**b) Ensure the input images are of the correct size (150x150) and preprocessed appropriately to match VGG16’s requirements-**

In [8]:
# Verify image shape and range
for image, label in train.take(1):
    print("Image shape:", image.shape)
    print("Pixel range:", tf.reduce_min(image).numpy(), "to", tf.reduce_max(image).numpy())

# Confirm batches created
print("Train batches:", train_batches)
print("Test batches:", test_batches)

Image shape: (150, 150, 3)
Pixel range: 0.0 to 1.0
Train batches: <_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
Test batches: <_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


###**2) Data Preprocessing and Augmentation-**

####**Implement image data generators for real-time augmentation-**

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply augmentation on-the-fly (rotation, shift, flip)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Note: Here we already used tfds dataset, so datagen is for demonstration
print("Data augmentation setup complete.")


Data augmentation setup complete.


###**3) Build and Train the Classification Model-**

####**Add custom fully connected layers on top of the VGG16 model for classification-**

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

# Add new layers on top of VGG16
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

print("Custom classification layers added.")


Custom classification layers added.


####**Freeze the convolutional layers of VGG16 and train only the custom layers-**

In [12]:
# Compile model
model.compile(
    optimizer=SGD(learning_rate=0.001, momentum=0.9),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train model
history = model.fit(
    train_batches,
    epochs=5,
    validation_data=test_batches
)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 103s 155ms/step - accuracy: 0.7708 - loss: 0.4574 - val_accuracy: 0.8917 - val_loss: 0.2623
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 80s 135ms/step - accuracy: 0.8778 - loss: 0.2878 - val_accuracy: 0.9043 - val_loss: 0.2367
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 80s 136ms/step - accuracy: 0.8854 - loss: 0.2631 - val_accuracy: 0.9058 - val_loss: 0.2295
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 80s 135ms/step - accuracy: 0.8926 - loss: 0.2490 - val_accuracy: 0.9091 - val_loss: 0.2156
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 82s 135ms/step - accuracy: 0.8961 - loss: 0.2341 - val_accuracy: 0.9093 - val_loss: 0.2173


###**4) Evaluate and Test the Model-**

In [13]:
val_loss, val_acc = model.evaluate(test_batches)
print(f"Validation Accuracy: {val_acc*100:.2f}%  |  Validation Loss: {val_loss:.4f}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 16s 106ms/step - accuracy: 0.9137 - loss: 0.2056
Validation Accuracy: 90.93%  |  Validation Loss: 0.2173


###**5) Save and Reload the Model-**

In [15]:
from tensorflow.keras.models import load_model

model.save("vgg16_cats_dogs.keras")
print("Model saved successfully!")

# Reload and verify
loaded_model = load_model("vgg16_cats_dogs.keras")
loss, acc = loaded_model.evaluate(test_batches)
print(f"Loaded Model Accuracy: {acc*100:.2f}%  |  Loss: {loss:.4f}")


Model saved successfully!
146/146 ━━━━━━━━━━━━━━━━━━━━ 18s 116ms/step - accuracy: 0.9137 - loss: 0.2056
Loaded Model Accuracy: 90.93%  |  Loss: 0.2173


###**Conclusion-**


The transfer learning model using VGG16 successfully classified cat and dog images.
It achieved around 91% accuracy with minimal loss (~0.21), showing strong generalization.
Pre-trained convolutional layers effectively extracted image features,
while the custom dense layers performed accurate classification.
The saved model was reloaded successfully, maintaining consistent performance.
